# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [16]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [17]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [18]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [19]:
def crop_center(img,croph,cropw):
    h,w,c = img.shape
    starth = h//2-(croph//2)
    startw = w//2-(cropw//2)    
    return img[starth:starth+croph,startw:startw+cropw,:]

In [20]:
def normalize_img(img):
    img1 = img - np.min(img)/np.max(img) - np.min(img)
    return img1

In [21]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    print('length of folder list = ', len(folder_list))
    
    #img_idx = [0,3,5,7,10,13,15,18,20,23,27] #create a list of image numbers you want to use for a particular video
    img_idx = [7,8,9,10,11,12,13,14,15,19,20,21,22,23,24]
    x = len(img_idx)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            #y, z, channel = imread.shape 
            batch_data = np.zeros((batch_size,x,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #print("before :", image.shape)
                    image = crop_center(image, 100, 100)
                    image = np.resize(image, (100, 100, 3))
                    #print("after :", image.shape)
                    batch_data[folder,idx,:,:,0] = normalize_img(image)[:, :, 0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = normalize_img(image)[:, :, 1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalize_img(image)[:, :, 2] #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [22]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [23]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

img_rows = 100
img_cols = 100
input_shape = (15, img_rows, img_cols, 3)



In [24]:
#write your model here
#Model1
#model = Sequential()
#model.add(Conv3D(32, kernel_size=(3, 3, 1),strides = (1,1,1), padding = 'same', activation='relu',input_shape=input_shape, data_format = 'channels_last'))
#model.add(MaxPooling3D(pool_size=(2, 2, 1)))
#model.add(Conv3D(64, kernel_size=(3, 3, 1),activation='relu'))
#model.add(MaxPooling3D(pool_size=(2, 2, 1)))
#model.add(Flatten())
#model.add(Dense(256, activation='relu'))
#model.add(Dense(5, activation='softmax'))


In [25]:
#write your model here
#model 2
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), padding = 'same', activation = 'relu',input_shape=input_shape, data_format = 'channels_last'))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides = (1,2,2)))
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation = 'relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides = (1,2,2)))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation = 'relu'))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation = 'relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides = (1,2,2)))
model.add(Conv3D(128, kernel_size=(2, 2, 2), activation = 'relu'))
model.add(Conv3D(128, kernel_size=(2, 2, 2), activation = 'relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides = (1,2,2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [26]:
optimiser = 'SGD' #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_3 (Conv3D)            (None, 15, 100, 100, 32)  2624      
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 15, 50, 50, 32)    0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 13, 48, 48, 32)    27680     
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 13, 24, 24, 32)    0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 11, 22, 22, 64)    55360     
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 9, 20, 20, 64)     110656    
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 9, 10, 10, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [27]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [28]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, min_lr=0.001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [29]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [30]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
length of folder list =  100
Source path =  ./Project_data/train ; batch size = 10
length of folder list =  663
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


67/67 [==============================] - 30s 449ms/step - loss: 12.8874 - categorical_accuracy: 0.1985 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2019-09-1817_47_52.055298/model-00001-12.88743-0.19851-12.41093-0.23000.h5
Epoch 2/10
67/67 [==============================] - 29s 431ms/step - loss: 12.7982 - categorical_accuracy: 0.2060 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_2019-09-1817_47_52.055298/model-00002-12.79825-0.20597-12.41093-0.23000.h5
Epoch 3/10
67/67 [==============================] - 29s 431ms/step - loss: 12.7982 - categorical_accuracy: 0.2060 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to model_init_2019-09-1817_47_52.055298/model-00003-12.79825-0.20597-12.41093-0.23000.h5
Epoch 4/10
67/67 [==============================] - 29s 433ms/step - loss: 12.8223 - categorical_accuracy: 0.2045 - val_loss: 12.4109 - val_categorical_accu